<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/LangChain_Expression_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain的新特性 - Expression Language

`LangChain Expression Language` 是一种以声明式方法，轻松地将链或组件组合在一起的机制。通过利用管道操作符，构建的任何链将自动具有完整的同步、异步和流式支持。

## Python管道 - Pipe

`Python` 的 `Pipe` 提供了管道实现。请参考 [https://github.com/JulienPalard/Pipe](https://github.com/JulienPalard/Pipe)。 来看几个例子

### 安装

In [3]:
# !pip install -U pipe

### 最简单一个例子

In [4]:
from pipe import select, where

numbers = [1, 2, 3, 4, 5]
result = list(numbers | where(lambda x: x % 2 == 0) | select(lambda x: x * 2))

result

[4, 8]

### 小小进阶

自定义管道 `uppercase` - 接受一个 `iterable` 参数

In [5]:
from pipe import Pipe

uppercase = Pipe(lambda iterable: (x.lower() for x in iterable))

words = ['red', 'green', 'blue', 'YELLOW']

uppercase_words = list(words | uppercase)

uppercase_words

['red', 'green', 'blue', 'yellow']

## LangChain Expression Language与管道

`LEL` 通过管道定义操作序列，帮助程序员以更加优雅简洁的编码方式构建功能逻辑。我们来看看如何通过表达式来重构几个经典的LangChain实例。

### 安装

我们需要安装最新版本的 `langchain` 以确保具有 `LEL` 功能的支持。

In [6]:
# !pip install -q -U langchain openai

### 提示词模版与模型


In [7]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


# openai.api_key  = os.environ['OPENAI_API_KEY']

#### 提示词模板与模型的传统用法

In [14]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage
)
from langchain.chains import LLMChain
from langchain.llms import OpenAI 

# llm = 

human_template="Show me the HEX code of color {color_name}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=OpenAI(), prompt=chat_prompt)

chain.run("RED")

'\n\nHEX code for the color red is #FF0000.'

#### 通过 `LEL` 连接提示词模板与模型

In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Show me the HEX code of color {color_name}")

chain = prompt | model

chain.invoke({"color_name": "RED"})

AIMessage(content='The HEX code of the color red is #FF0000.', additional_kwargs={}, example=False)

### 一个稍稍复杂的例子

现在我们给刚才搭建的管道追加一些环节

#### 添加标准输出解析

In [17]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | model | StrOutputParser()

chain.invoke({"color_name": "RED"})

'The HEX code for the color red is #FF0000.'

#### 添加函数调用

我们来给管道中的模型添加一些函数调用。注，我们并不真正调用函数，只解析出函数调用的数据。

`JsonOutputFunctionsParser` 用来将函数调用的回复解析为JSON格式，请参考[API 文档](https://api.python.langchain.com/en/latest/output_parsers/langchain.output_parsers.openai_functions.JsonKeyOutputFunctionsParser.html)

In [22]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

functions = [
    {
      "name": "save_color_code",
      "description": "Save the HEX code of color and its name",
      "parameters": {
        "type": "object",
        "properties": {
          "hex_code": {
            "type": "string",
            "description": "The HEX code of the color"
          },
          "color": {
            "type": "string",
            "description": "The color name"
          }
        },
        "required": ["hex_code", "color"]
      }
    }
  ]
chain = prompt | model.bind(function_call = {"name": "save_color_code"}, functions = functions) | JsonOutputFunctionsParser()

chain.invoke({"color_name": "RED"})
chain.invoke({"input": "hi","color_name": "RED"})

{'hex_code': '#FF0000', 'color': 'RED'}